# Wstęp do Sztucznej Inteligencji - rok akademicki 2022/2023

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia.

# Temat: Prolog - Programowanie w logice cz. III
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.

## Przykład na dobry początek
Pięcioro przyjaciół rywalizowało na bieżni. Wincenty ze smutkiem opowiedział, że mu się nie udało zająć pierwszego miejsca. Grzegorz przybiegł na metę jako trzeci i był za Dymitrem. Wincenty zauważył, że Dymitr nie zajął drugiego miejsca, a
Andrzej nie był ani pierwszym ani ostatnim. Borys powiedział, że przybiegł na metę w ślad za Wincentym.

Pytanie: kto przybiegł na jakim miejscu?

Przykład realizacji w Prologu.

```prolog
miejsce(W,D,A,B,G) :-
    L0 = [1,2,3,4,5],
    G = 3, /* Grzegorz zajął trzecie miejsce */
    select(W, L0, L1), W \= 1, /* Wincenty nie zajął pierwszego miejsca */
    select(D, L1, L2), D \= 2, /* Dymitr nie zajął drugiego, ani tego co Wincenty */
    select(A, L2, L3), A \= 1, A \= 5, /* Andrzej nie zajął pierwszego ani piątego, ani tego co Wincenty lub Dymitr*/
    select(B, L3, [G]), /* pozostale miejsca zajęli Borys i Grzegorz */
    B =:= W+1, /* Borys przybiegl zaraz za Wincentym */
    G > D. /* Dymitr był na mecie wcześniej niż Grzegorz */
```

## Wnioskowanie w przód i w tył w PROLOGu
Dwoma podstawowymi metodami wnioskowania wykorzystywanymi w systemach ekspertowych regułowych jest wnioskowanie w przód oraz wnioskowanie wstecz.

Załóżmy, że w bazie wiedzy posiadamy następujące reguły:
```prolog
R1: IF A AND C THEN E
R2: IF F AND B THEN A
R3: IF A AND G AND B THEN F
R4: IF C THEN B
R5: IF E AND G AND F THEN D
R6: IF A AND G THEN C
```
Załóżmy również, że prawdziwe (znane) są fakty A, G, C.

Wnioskowanie w przód polega na tym, by na podstawie znanych faktów oraz posiadanych reguł wnioskować o prawdziwości pozostałych faktów, wszystkich, jakie tylko wynikają w sposób logiczny z posiadanej wiedzy.

W powyższym przykładzie prawdziwe są A, G, C. Zakładając, że przeglądamy bazę wiedzy od reguły R1 do R6, możemy przeprowadzić następujące wnioskowanie.

- Posługując się regułą `R1` dowodzimy prawdziwości `E`, gdyż część warunkowa `R1` jest spełniona - prawdziwe są fakty `A` i `C`. Znane obecnie fakty to `A`, `G`, `C`, `E`.
- `R2` we wniosku ma `A`, który już jest znany, więc `R1` nie wnosi nic nowego do naszej wiedzy. Warto zwrócić uwagę, że z tego, iż prawdziwe jest `A`, nie możemy wnioskować o prawdziwości `F` i `B`.
- Nie wszystkie wymagane fakty z części warunkowej `R3` są znane (nie jest znane `B`), więc `F` nie jest dowiedzione (możliwe, że będzie dowiedzione później, gdy zostaną dowiedzione nowe fakty).
- Na podstawie `R4` oraz znanego `C`, dowodzimy `B`. Znane fakty to `A`, `G`, `C`, `E`, `B`.
- `R5` nie może być obecnie użyta - nie jest znane `F` wymagane w części warunkowej.
- `R6` nie wnosi nowej wiedzy.
- Zaczynamy przeglądać bazę reguł od początku pomijając już wykorzystane reguły. 
- Ze znanych obecnie faktów oraz reguły `R3` dowodzimy prawdziwości `F`. Znane obecnie fakty to `A`, `G`, `C`, `E`, `B`, `F`.
- Reguła `R5` pozwala teraz dowieść prawdziwości `D`. Znane fakty to `A`, `G`, `C`, `E`, `B`, `F`, `D`.
- Wnioskowanie w przód kończy się, jako że nie można już wywnioskować nic nowego - wszystkie możliwe fakty zostały dowiedzione i żadna z reguł nie może już być użyta.

Wnioskowanie w tył polega na próbie dowiedzenia pewnej hipotezy, np. prawdziwości faktu `D`. Nie chodzi zatem o wykazanie prawdziwości wszystkiego, co tylko wynika logicznie z posiadanej wiedzy, a jedynie tego, co jest potrzebne by dowieść prawdziwości głównej hipotezy. Spróbujmy dowieść prawdziwości faktu `D` za pomocą wnioskowania w tył. Znane fakty to `A`, `G`, `C`.
- `D` nie ma wśród znanych faktów, przeglądane są więc reguły. Widać, że `D` znajduje się we wniosku reguły `R5`. By była ona spełniona, znane muszą być `E`, `G` i `F`. Znane jest jedynie `G`, następuje zatem próba dowiedzenia `E` oraz `F`.
  - `E` występuje we wniosku `R1`. Spełniona jest część warunkowa `R1` (znane są `A` i `C`) zatem `E` jest dowiedzione. Znane fakty to `A`, `G`, `C`, `E`.
  - `F` występuje we wniosku reguły `R3`. Aby dowieść `F` znane muszą być `A`, `G` i `B`, spośród których nie jest znane `B`. Następuje próba dowiedzenia `B`.
    - `B` występuje we wniosku reguły `R4`, która jest spełniona (gdyż znane jest `C`). `B` zostało dowiedzione. Znane fakty to `A`, `G`, `C`, `E`, `B`.
  - `A`, `G`, `B` są znane, zatem z reguły `R3` wynika prawdziwość `F`. Znane fakty to `A`, `G`, `C`, `E`, `B`, `F`.
- Następuje powrót do reguły `R5`. `E`, `G` i `F` są teraz znane, zatem z `R5` wynika `D`, co było główną hipotezą, którą udało się dowieść. Znane fakty to `A`, `G`, `C`, `E`, `B`, `F` i `D`.

## REALIZACJA W PROLOGU

### Przykład 1.
Stwórz plik `przyklad1.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- fakt(a), fakt(c).
regula(a):- fakt(f), fakt(b).
regula(f):- fakt(a), fakt(g), fakt(b).
regula(b):- fakt(c).
regula(d):- fakt(e), fakt(g), fakt(f).
regula(c):- fakt(a), fakt(g).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g).
```

__Uwaga:__ `:- dynamic` informuje, że definicje predykatów, które następują po `dynamic`, mogą ulegać zmianom w trakcie wykonania programu.

#### Wykonaj zapytania.
Wnioskowanie w przód:
```prolog
?- sprawdz(X).
```
Wnioskowanie w tył:
```prolog
?- sprawdz(d).
```
Widać, że w ten sposób wnioskowanie w przód nie przebiega tak jak powinno - nie wszystkie fakty możliwe do wywnioskowania zostały dowiedzione. Również wnioskowanie w tył nie jest poprawnie zrealizowane. Problem polega na tym, iż dowiedzione fakty nie są wprowadzane do bazy wiedzy PROLOGA w celu ich wykorzystania w dalszym wnioskowaniu.

Baza znanych faktów zostaje przeszukana w takiej kolejności w jakiej fakty zostały wprowadzone i jeśli napotkany zostanie fakt, o który pytamy, odpowiedź będzie pozytywna, a zatem prawdziwość faktu zostanie potwierdzona. Jeśli taki fakt nie zostanie znaleziony, Prolog odpowiada „nie”. „Nie” w Prologu nie oznacza, ze coś jest nieprawdziwe, a raczej, że Prolog na podstawie znanych mu faktów (i reguł) nie jest w stanie go potwierdzić.

__Uwaga__: Przy przejściu do kolejnego przykładu należy "wyczyścić aktualny stan wiedzy". tzn. wykonać polecenia
```prolog
unload_file("przykladX.pl").
retractall(fakt(_)).
```
Lub wyjść z konsoli poleceniem `halt.` i rozpocząć od nowa.

### Przykład 2.
Stwórz plik `przyklad2.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- fakt(a), fakt(c), assert(fakt(e)).
regula(a):- fakt(f), fakt(b), assert(fakt(a)).
regula(f):- fakt(a), fakt(g), fakt(b), assert(fakt(f)).
regula(b):- fakt(c), assert(fakt(b)).
regula(d):- fakt(e), fakt(g), fakt(f), assert(fakt(d)).
regula(c):- fakt(a), fakt(g), assert(fakt(c)).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g).
```
#### Wykonaj zapytania.
Wnioskowanie w przód:
```prolog
?- sprawdz(X).
```
```prolog
?- sprawdz(X).
```
Wszystkie fakty zostały dowiedzione, ale konieczne były dwa zapytania, by dowieść wszystkich faktów. Każdy dowiedziony przez regułę fakt, jest dodawany do bazy wiedzy predykatem `assert`.

Wnioskowanie w tył:
```prolog
?- sprawdz(d).
```
Wnioskowanie w tył nadal nie działa. Reguły nie są stanie wywoływać siebie nawzajem w razie potrzeby.

### Przykład 3.
__Uwaga! Uruchomienie tego przykładu powoduje wystąpienie nieskończonej rekurencji!__

Stwórz plik `przyklad3.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- sprawdz(a), sprawdz(c).
regula(a):- sprawdz(f), sprawdz(b).
regula(f):- sprawdz(a), sprawdz(g), sprawdz(b).
regula(b):- sprawdz(c).
regula(d):- sprawdz(e), sprawdz(g), sprawdz(f).
regula(c):- sprawdz(a), sprawdz(g).
sprawdz(a):-fakt(a) ; regula(a).
sprawdz(b):-fakt(b) ; regula(b).
sprawdz(c):-fakt(c) ; regula(c).
sprawdz(d):-fakt(d) ; regula(d).
sprawdz(e):-fakt(e) ; regula(e).
sprawdz(f):-fakt(f) ; regula(f).
sprawdz(g):-fakt(g) ; regula(g).
```
#### Wykonaj zapytania.
Wnioskowanie w przód:
```prolog
?- sprawdz(X).
```
Przykład ten jest próbą takiego zapisania reguł, by każda z nich nie sprawdzała prawdziwości potrzebnych jej faktów bezpośrednio za pomocą predykatu `fakt`, lecz raczej za pomocą predykatu `sprawdz`, co pozwoli na pośrednie wzajemne wywoływanie siebie przez reguły. Jednak zapisanie w ten sposób prowadzi to do wystąpienie nieskończonej rekurencji.

### Przykład 4.
Stwórz plik `przyklad4.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
:-dynamic inuse/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- not(inuse(e)), assert(inuse(e)), sprawdz(a), sprawdz(c), retract(inuse(e)).
regula(a):- not(inuse(a)), assert(inuse(a)), sprawdz(f), sprawdz(b), retract(inuse(a)).
regula(f):- not(inuse(f)), assert(inuse(f)), sprawdz(a), sprawdz(g), sprawdz(b), retract(inuse(f)).
regula(b):- not(inuse(b)), assert(inuse(b)), sprawdz(c), retract(inuse(b)).
regula(d):- not(inuse(d)), assert(inuse(d)), sprawdz(e), sprawdz(g), sprawdz(f), retract(inuse(d)).
regula(c):- not(inuse(c)), assert(inuse(c)), sprawdz(a), sprawdz(g), retract(inuse(c)).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g).
```
Pomocniczy predykat `inuse` zapobiega wystąpieniu zapętlenia. Warto zaznaczyć, iż w tej wersji nowe fakty nie są zapamiętywane (w celu przejrzystości przykładu).
#### Wykonaj zapytania.
Wnioskowanie w przód:
```prolog
?- sprawdz(X).
```
```prolog
?- fakt(X).
```
Wszystkie fakty zostały dowiedzione, jednak nie zostały zapamiętane.

Wnioskowanie w tył również działa poprawnie:
```prolog
?- sprawdz(d).
```

### Przykład 5.
Połączmy poprzedni przykład z możliwością zapamiętywania dowiedzionych faktów.

Zwróć uwagę na użycie `(retract(inuse(e)), fail).`. Celem jest usunięcie predykatu `inuse(.)` zawsze, nawet jeśli pierwsza część alternatywy (przed `;`) zawiedzie przed wywołaniem `retract(.)`.

Stwórz plik `przyklad5.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
:-dynamic inuse/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- not(inuse(e)), assert(inuse(e)), (sprawdz(a), sprawdz(c), assert(fakt(e)), retract(inuse(e))) ;
            (retract(inuse(e)), fail).
regula(a):- not(inuse(a)), assert(inuse(a)), (sprawdz(f), sprawdz(b), assert(fakt(a)), retract(inuse(a))) ;
            (retract(inuse(a)), fail).
regula(f):- not(inuse(f)), assert(inuse(f)), (sprawdz(a), sprawdz(g), sprawdz(b), assert(fakt(f)),
            retract(inuse(f))) ; (retract(inuse(f)), fail).
regula(b):- not(inuse(b)), assert(inuse(b)), (sprawdz(c), assert(fakt(b)), retract(inuse(b))) ;
            (retract(inuse(b)), fail).
regula(d):- not(inuse(d)), assert(inuse(d)), (sprawdz(e), sprawdz(g), sprawdz(f), assert(fakt(d)),
            retract(inuse(d))) ; (retract(inuse(d)), fail).
regula(c):- not(inuse(c)), assert(inuse(c)), (sprawdz(a), sprawdz(g), assert(fakt(c)), retract(inuse(c))) ;
            (retract(inuse(c)), fail).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g).
```
#### Wykonaj zapytania.
```prolog
?- sprawdz(X).
```
```prolog
?- fakt(X).
```
Fakty zostały zapamiętane. 

Działa również wnioskowanie w tył:
```prolog
?- sprawdz(d).
```
```prolog
?- fakt(X).
```
Również w tym przypadku fakty zostały wprowadzone do bazy wiedzy PROLOGA (jednak w innej kolejności niż przy wnioskowanie w przód).

### Przykład 6.
Do naszego programu dodamy możliwość zapytania się przez PROGLOGA o prawdziwość danego faktu.

Stwórz plik `przyklad6.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
:-dynamic inuse/1.
% fakt(a).
% fakt(g).
% fakt(c).
regula(e):- not(inuse(e)), assert(inuse(e)), (sprawdz(a), sprawdz(c), assert(fakt(e)), retract(inuse(e))) ;
            (retract(inuse(e)), fail).
regula(a):- not(inuse(a)), assert(inuse(a)), (sprawdz(f), sprawdz(b), assert(fakt(a)), retract(inuse(a))) ;
            (retract(inuse(a)), fail).
regula(f):- not(inuse(f)), assert(inuse(f)), (sprawdz(a), sprawdz(g), sprawdz(b), assert(fakt(f)),
            retract(inuse(f))) ; (retract(inuse(f)), fail).
regula(b):- not(inuse(b)), assert(inuse(b)), (sprawdz(c), assert(fakt(b)), retract(inuse(b))) ; 
            (retract(inuse(b)), fail).
regula(d):- not(inuse(d)), assert(inuse(d)), (sprawdz(e), sprawdz(g), sprawdz(f), assert(fakt(d)),
            retract(inuse(d))) ; (retract(inuse(d)), fail).
regula(c):- not(inuse(c)), assert(inuse(c)), (sprawdz(a), sprawdz(g), assert(fakt(c)), retract(inuse(c))) ;
            (retract(inuse(c)), fail).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a); not(fakt(a)), zapytaj(a), assert(fakt(a)).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b); not(fakt(b)), zapytaj(b), assert(fakt(b)).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c); not(fakt(c)), zapytaj(c), assert(fakt(c)).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d); not(fakt(d)), zapytaj(d), assert(fakt(d)).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e); not(fakt(e)), zapytaj(e), assert(fakt(e)).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f); not(fakt(f)), zapytaj(f), assert(fakt(f)).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g); not(fakt(g)), zapytaj(g), assert(fakt(g)).
zapytaj(X):- write('czy '), write(X), write(' jest prawdziwe?[t/n] '), nl, read(Odp), Odp='t'.
```
#### Wykonaj zapytania.
```prolog
?- sprawdz(d).
```
```prolog
czy a jest prawdziwe?[t/n] t.
czy g jest prawdziwe?[t/n] n.
czy f jest prawdziwe?[t/n] t.
czy g jest prawdziwe?[t/n] n.
czy c jest prawdziwe?[t/n] t.
czy e jest prawdziwe?[t/n] t.
czy g jest prawdziwe?[t/n] t.
```
Wielokrotne pytanie o to samo jest irytujące, lecz problem ten rozwiązany jest w następnej wersji programu.

### Przykład 7.
Ostateczna (prawie) wersja.

Stwórz plik `przyklad7.pl` o następującej treści:
```prolog
:-dynamic fakt/1.
:-dynamic inuse/1.
:-dynamic asked/1.
fakt(a).
fakt(g).
fakt(c).
regula(e):- not(inuse(e)), assert(inuse(e)), (sprawdz(a), sprawdz(c), assert(fakt(e)), retract(inuse(e))) ;
            (retract(inuse(e)), fail).
regula(a):- not(inuse(a)), assert(inuse(a)), (sprawdz(f), sprawdz(b), assert(fakt(a)), retract(inuse(a))) ;
            (retract(inuse(a)), fail).
regula(f):- not(inuse(f)), assert(inuse(f)), (sprawdz(a), sprawdz(g), sprawdz(b), assert(fakt(f)),
            retract(inuse(f))) ; (retract(inuse(f)), fail).
regula(b):- not(inuse(b)), assert(inuse(b)), (sprawdz(c), assert(fakt(b)), retract(inuse(b))) ;
            (retract(inuse(b)), fail).
regula(d):- not(inuse(d)), assert(inuse(d)), (sprawdz(e), sprawdz(g), sprawdz(f), assert(fakt(d)),
            retract(inuse(d))) ; (retract(inuse(d)), fail).
regula(c):- not(inuse(c)), assert(inuse(c)), (sprawdz(a), sprawdz(g), assert(fakt(c)), retract(inuse(c))) ;
            (retract(inuse(c)), fail).
sprawdz(a):- fakt(a) ; not(fakt(a)), regula(a); not(fakt(a)), zapytaj(a), assert(fakt(a)).
sprawdz(b):- fakt(b) ; not(fakt(b)), regula(b); not(fakt(b)), zapytaj(b), assert(fakt(b)).
sprawdz(c):- fakt(c) ; not(fakt(c)), regula(c); not(fakt(c)), zapytaj(c), assert(fakt(c)).
sprawdz(d):- fakt(d) ; not(fakt(d)), regula(d); not(fakt(d)), zapytaj(d), assert(fakt(d)).
sprawdz(e):- fakt(e) ; not(fakt(e)), regula(e); not(fakt(e)), zapytaj(e), assert(fakt(e)).
sprawdz(f):- fakt(f) ; not(fakt(f)), regula(f); not(fakt(f)), zapytaj(f), assert(fakt(f)).
sprawdz(g):- fakt(g) ; not(fakt(g)), regula(g); not(fakt(g)), zapytaj(g), assert(fakt(g)).
zapytaj(X):- not(asked(X)), assert(asked(X)), nl, write('czy '), write(X), write(' jest prawdziwe?[t/n] '),                      nl, read(Odp), Odp='t'.
reset:- retractall(fakt(_)), retractall(asked(_)).
dodaj(X):- not(fakt(X)), assert(fakt(X)).
wypisz_fakty:- not(fakt(_)), write('Nie ma znanych faktow!'), nl, !.
wypisz_fakty:- fakt(X), write(X), write(' '), fail.
wypisz_fakty.
wnioskuj_w_przod:- nl, write('Fakty znane przed wnioskowaniem: '), nl, wypisz_fakty, fail.
wnioskuj_w_przod:- sprawdz(_), fail.
wnioskuj_w_przod:- nl, write('Fakty znane po wnioskowaniu: '), nl, wypisz_fakty.
wnioskuj_w_tyl(_):- nl, write('Fakty znane przed wnioskowaniem: '), nl, wypisz_fakty, fail.
wnioskuj_w_tyl(X):- sprawdz(X), nl, write(X), write(' dowiedzione'), nl, 
                    write('Fakty znane po wnioskowaniu: '), nl,
wypisz_fakty, !.
wnioskuj_w_tyl(X):- nl, write(X), write(' nie zostalo dowiedzione'), nl, 
                    write('Fakty znane po wnioskowaniu: '), nl,
wypisz_fakty.
powitanie:- write('Witaj!'),nl, nl.
```
Przykładowa sesja: niedowiedzione
```prolog
?- wypisz_fakty.
```
```prolog
?- wnioskuj_w_przod.
```
Fakty znane przed wnioskowaniem: `a` `g` `c`

Fakty znane po wnioskowaniu: `a` `g` `c` `b` `e` `f` `d`
```prolog
?- reset.
```
```prolog
?- wypisz_fakty.
```
Nie ma znanych faktow!
```prolog
?- dodaj(a), dodaj(b), dodaj(c).
```
```prolog
?- wnioskuj_w_tyl(f).
```
Fakty znane przed wnioskowaniem: `a` `b` `c`
```prolog
czy g jest prawdziwe?[t/n] t.
```
f dowiedzione

Fakty znane po wnioskowaniu: `a` `b` `c` `g` `f`

Ta wersja ma pewną wadę:
```prolog
?- reset.
```
```prolog
?- wnioskuj_w_przod.
```
Fakty znane przed wnioskowaniem:

Nie ma znanych faktow!
```prolog
czy a jest prawdziwe?[t/n] t.
czy g jest prawdziwe?[t/n] t.
```
Fakty znane po wnioskowaniu: `a` `g` `c` `b` `f` `a` `e` `d` `e`

Niektóre fakty zostały wprowadzone niepotrzebnie więcej niż raz. Co trzeba zmienić by tak się nie działo? (Zwróć uwagę na wykorzystanie predykatu `dodaj`).

## Generowanie wersji samodzielnej aplikacji

Stwórz plik `hello.pl` o następującej treści:
```prolog
hello :- format('Hello world!~n'), halt.
```
W celu wygenerowania wersji samodzielnej (pliku wykonywalnego) naszego programu, wykonaj w terminalu
```bash
swipl -o myexe -g hello -c hello.pl
```
gdzie poszczególne opcje:
- `-o` to `output`, 
- `-g` to `goal` czyli reguła, która ma się wykonać przy starcie programu,
- `-c` to plik z bazą wiedzy.

Po kompilacji możemy wykonać nasz program tak jak każdy inny plik wykonywalny.
```bash
./myexe
```
W przypadku komunikatu a braku praw, należy uruchomić poleceniem
```bash
bash myexe
```

Aby nasz program działał na innym komputerze bez koniczności instalacji SWI-Prolog należy dołączyć również odpowiednią bibliotekę, szczegóły: http://www.swi-prolog.org/FAQ/UnixExe.html.

## Przykład: Animals

Poniższy przykładowy mini system ekspertowy składa się z bazy wiedzy w pliku `animals_bw.pl` oraz głównego programu `animals.pl`. Próbuje on zganąć jakie zwierzę użytkownik ma na myśli. 

Zwróć uwagę na niedoskonałości programu. Przykładowo, program niepotrzebnie pyta wiele razy o to samo. Nie może również uczyć się nowych zwierząt.

Można go skompilować za pomocą polecenia (wersja na Windows):

`..\..\bin\plcon.exe --goal=start --stand_alone=true -o animals.exe -c animals.pl`

#### Zawartość pliku `animals_bw.pl`:

```prolog
zwierze(gepard):-
				jest_to(ssak),
				jest_to(miesozerny),
				pozytywne(ma,brunatny_kolor),
				pozytywne(ma, ciemne_plamy).

zwierze(tygrys):-
				jest_to(ssak),
				jest_to(miesozerny),
				pozytywne(ma,brunatny_kolor),
				pozytywne(ma, czarne_pasy).
				
zwierze(zyrafa):-
				jest_to(kopytonogi),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, ciemne_plamy).
				
zwierze(zebra):-
				jest_to(kopytonogi),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, czarne_pasy).
				
zwierze(strus):-
				jest_to(ptak),
				negatywne(czy, lata),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, biale_i_czarne_kolory).

zwierze(pingwin):-
				jest_to(ptak),
				negatywne(czy, lata),
				pozytywne(czy, plywa),
				pozytywne(ma, biale_i_czarne_kolory).

zwierze(albatros):-
				jest_to(ptak),
				pozytywne(czy, lata_dobrze).
			
jest_to(ssak):-
				pozytywne(ma,siersc).				

jest_to(ssak):-
				pozytywne(czy,daje_mleko).

jest_to(ptak):-
				pozytywne(ma,piora).

jest_to(ptak):-
				pozytywne(czy,lata),
				pozytywne(czy, znosi_jaja).
				
jest_to(miesozerny):-
					pozytywne(czy,je_mieso).				
				
jest_to(miesozerny):-
					pozytywne(ma,ostre_zeby),
					pozytywne(ma,pazury),
					pozytywne(ma, wysuniete_do_przodu_oczy).
					
jest_to(kopytonogi):-
					jest_to(ssak),
					pozytywne(ma, kopyta).
					
jest_to(kopytonogi):-
					jest_to(ssak),
					pozytywne(czy, przezuwa_pokarm).
                    
zwierze(gepard):-
				jest_to(ssak),
				jest_to(miesozerny),
				pozytywne(ma,brunatny_kolor),
				pozytywne(ma, ciemne_plamy).
				
zwierze(tygrys):-
				jest_to(ssak),
				jest_to(miesozerny),
				pozytywne(ma,brunatny_kolor),
				pozytywne(ma, czarne_pasy).
				
zwierze(zyrafa):-
				jest_to(kopytonogi),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, ciemne_plamy).
				
zwierze(zebra):-
				jest_to(kopytonogi),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, czarne_pasy).
				
zwierze(strus):-
				jest_to(ptak),
				negatywne(czy, lata),
				pozytywne(ma, dluga_szyja),
				pozytywne(ma, dlugie_nogi),
				pozytywne(ma, biale_i_czarne_kolory).

zwierze(pingwin):-
				jest_to(ptak),
				negatywne(czy, lata),
				pozytywne(czy, plywa),
				pozytywne(ma, biale_i_czarne_kolory).

zwierze(albatros):-
				jest_to(ptak),
				pozytywne(czy, lata_dobrze).
				
jest_to(ssak):-
				pozytywne(ma,siersc).				

jest_to(ssak):-
				pozytywne(czy,daje_mleko).

jest_to(ptak):-
				pozytywne(ma,piora).

jest_to(ptak):-
				pozytywne(czy,lata),
				pozytywne(czy, znosi_jaja).
				
jest_to(miesozerny):-
					pozytywne(czy,je_mieso).				
				
jest_to(miesozerny):-
					pozytywne(ma,ostre_zeby),
					pozytywne(ma,pazury),
					pozytywne(ma, wysuniete_do_przodu_oczy).
					
jest_to(kopytonogi):-
					jest_to(ssak),
					pozytywne(ma, kopyta).
					
jest_to(kopytonogi):-
					jest_to(ssak),
					pozytywne(czy, przezuwa_pokarm).
```


#### Zawartość pliku `animals.pl`:

```prolog
:-dynamic xpozytywne/2.
:-dynamic xnegatywne/2.

:-(clause(zwierze(_),_); consult('animals_bw.pl')).	

pozytywne(X,Y):-
				xpozytywne(X,Y),
				!.

pozytywne(X,Y):-
				not(xnegatywne(X,Y)),
				pytaj(X,Y,tak).

negatywne(X,Y):-
				xnegatywne(X,Y),
				!.
				
negatywne(X,Y):-
				not(xpozytywne(X,Y)),
				pytaj(X,Y,nie).
				
pytaj(X,Y,tak):-
				!,
				write(X),write(' to_zwierze '),write(Y),write(' ?(t/n)'),nl,
				read(Odp),
				Odp = t,
				assertz(xpozytywne(X,Y)).
				
pytaj(X,Y,nie):-
				!,
				write(X),write(' to_zwierze '),write(Y),write(' ?(t/n)'),nl,
				read(Odp),
				Odp = n,
				assertz(xnegatywne(X,Y)).
				
wyczysc_fakty:-
				retractall(xpozytywne(_,_)),
				retractall(xnegatywne(_,_)),
				write('fakty usuniete'),nl.
				
co_to_za_zwierze(X):-zwierze(X).				
				
start:-
		nl,nl,
		write('Sprobuje zgadnac o jakie zwierze chodzi!'),nl,
		co_to_za_zwierze(X),
		nl,nl,	
		write('to zwierze to '),
		write(X),
		write('!!!'),
		nl,nl,			
		write('Jeszcze raz? (t/n)'),nl,
		read(Odp),
		(Odp = t, wyczysc_fakty, start ; !, wyczysc_fakty, write('koniec!'), get0(_), halt).
		
start:-
		nl,nl,
		write('Niestety nie moge odgadnac :('),nl,
		write('Jeszcze raz? (t/n)'),nl,
		read(Odp),
		(Odp = t, wyczysc_fakty, start ; !, wyczysc_fakty, write('koniec!')).				

```

## Przykład: Piłkarze

Poniższy przykładowy program składa się z bazy wiedzy w pliku `pilkarze_bw.pl` oraz głównego programu `pilkarze.pl`. Program umożliwia zarządzanie małą bazą danych o piłkarzach. Predykat `zawodnik` przechowuje informacje o nazwiku, drużynie i wzroście.

Można go skompilować za pomocą polecenia (wersja na Windows):

`..\..\bin\plcon.exe --goal=menu --stand_alone=true -o pilkarze.exe -c pilkarze.pl`

#### Zawartość pliku `pilkarze_bw.pl`:

```prolog
:-dynamic zawodnik/3.
zawodnik(kowal, druzyna2, 189).
zawodnik(adamiec, druzyna2, 180).
zawodnik(socha, druzyna3, 191).
zawodnik(boniek, druzyna3, 192).
```

#### Zawartość pliku `pilkarze.pl`:

```prolog
:-(clause(zawodnik(_,_,_),_); consult('pilkarze_bw.pl')).	

menu:-
		write('********************************************************'),nl,
		write('*                                                      *'),nl,
		write('*                PILKARZE                              *'),nl,
		write('*                                                      *'),nl,
		write('********************************************************'),nl,
		write('*	1. - przeglad bazy                                *'),nl,
		write('*    2. - dodawanie zawodnikow                         *'),nl,
		write('*    3. - aktualizacja danych                          *'),nl,
		write('*    4. - usuwanie zawodnikow                          *'),nl,
		write('*    5. - koniec pracy                                 *'),nl,
		write('********************************************************'),nl,
		nl,nl,
		write('wybor (zakonczony kropka): '),
		read(X), 
		menu1(X).
		
menu1(1):-
			nl,nl,
			write('podaj nazwisko zawodnika (zakonczone kropka): '),nl,
			read(Nazwisko),nl,
			!,
			wyszukaj(Nazwisko),
			dalej(Odp),
			(Odp = t, menu1(1) ; menu).

			
menu1(2):-
			nl,nl,
			write('podaj nazwisko zawodnika:'),
			read(Nazwisko),nl,
			write('podaj druzyne zawodnika:'),
			read(Druzyna),nl,
			write('podaj wzrost:'),
			read(Wzrost),nl,
			(zawodnik(Nazwisko,Druzyna,Wzrost), write('taki zawodnik juz istnieje'), nl ;
			assertz(zawodnik(Nazwisko,Druzyna,Wzrost)), write('zawodnik dodany')),
			dalej(Odp),
			(Odp = t, menu1(2) ; menu).

menu1(3):-
			nl,nl,
			write('podaj nazwisko zawodnika:'),
			read(Nazwisko),nl,
			(retract(zawodnik(Nazwisko,Druzyna, Wzrost)),
			write('podaj nowa druzyne:'),
			read(Druzyna_Nowa),nl,
			write('podaj nowy wzrost:'),
			read(Wzrost_Nowy),nl,
			assertz(zawodnik(Nazwisko,Druzyna_Nowa,Wzrost_Nowy)),
			write('dane zostaly zmienione!');
			nl,
			write('brak zawodnika w bazie')),
			dalej(Odp),
			(Odp = t, menu1(3) ; menu).
			

menu1(4):-
			nl,nl,
			write('podaj nazwisko zawodnika:'),
			read(Nazwisko),nl,
			(retract(zawodnik(Nazwisko,_,_)),
			write('zawodnik usuniety');
			nl,
			write('brak zawodnika w bazie')),
			dalej(Odp),
			(Odp = t, menu1(4) ; menu).

menu1(5):-		
			zapis_bazy,
			nl,
			retractall(zawodnik(_,_,_)),
			write('koniec pracy...'),
			nl,
			get0(_),
			halt.

menu1(_):-	
			nl,nl,
			write('bledny numer!'),
			nl,
			menu.

wyszukaj(Nazwisko):-
					not(zawodnik(Nazwisko, Druzyna, Wzrost)),
					write('brak zawodnika o takim nazwisku'),nl,
					!.

wyszukaj(Nazwisko):-
					zawodnik(Nazwisko, Druzyna, Wzrost),
					write('druzyna:'), write(Druzyna),
					nl,
					write('wzrost:'),write(Wzrost).
					
dalej(Odp):-
			nl,
			write('kontynuacja? t/n'),nl,
			read(Odp).
			
zapis_bazy:-
			tell('pilkarze_bw.pl'),
			write(':-dynamic zawodnik/3.'),nl,
			zapis,
			told.
zapis:-		
		zawodnik(Nazwisko, Druzyna, Wzrost),
		write(zawodnik(Nazwisko,Druzyna,Wzrost)),
		write(.),
		nl,
		fail.		
		
zapis.				
```

&copy; Katedra Informatyki, Politechnika Krakowska